In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
sys.path.append('../../../../nadavp/opt_lev_analysis/scripts/calibration/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [6]:
step_cal_dir = r'/data/old_trap/20210405/bead1/discharge/before_neutrality/'
max_file = 500

In [14]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
len(step_cal_files)

106

In [8]:
# Do the step calibration
step_file_objs = []
for filname in step_cal_files[1:max_file]:
    df = bu.DataFile();
    df.load(filname);
    df.load_other_data()
    step_file_objs.append(df);

In [63]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [60]:
func1_scale(0)

<ValueView of Minuit at 3f6d03c8>
  A: -3.0278044505394064e-05
  f: 41.0
  phi: 0.0


14154.136079549795

In [23]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 106
1 / 106
2 / 106
3 / 106
4 / 106
5 / 106
6 / 106
7 / 106
8 / 106
9 / 106
10 / 106
11 / 106
12 / 106
13 / 106
14 / 106
15 / 106
16 / 106
17 / 106
18 / 106
19 / 106
20 / 106
21 / 106
22 / 106
23 / 106
24 / 106
25 / 106
26 / 106
27 / 106
28 / 106
29 / 106
30 / 106
31 / 106
32 / 106
33 / 106
34 / 106
35 / 106
36 / 106
37 / 106
38 / 106
39 / 106
40 / 106
41 / 106
42 / 106
43 / 106
44 / 106
45 / 106
46 / 106
47 / 106
48 / 106
49 / 106
50 / 106
51 / 106
52 / 106
53 / 106
54 / 106
55 / 106
56 / 106
57 / 106
58 / 106
59 / 106
60 / 106
61 / 106
62 / 106
63 / 106
64 / 106
65 / 106
66 / 106
67 / 106
68 / 106
69 / 106
70 / 106
71 / 106
72 / 106
73 / 106
74 / 106
75 / 106
76 / 106
77 / 106
78 / 106
79 / 106
80 / 106
81 / 106
82 / 106
83 / 106
84 / 106
85 / 106
86 / 106
87 / 106
88 / 106
89 / 106
90 / 106
91 / 106
92 / 106
93 / 106
94 / 106
95 / 106
96 / 106
97 / 106
98 / 106
99 / 106
100 / 106
101 / 106
102 / 106
103 / 106
104 / 106
105 / 106


IndexError: list index out of range

In [24]:
_,ax = plt.subplots(figsize=(9.5,4))
_ = ax.scatter(range(len(amps3)), amps3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
scale = 910

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-0.3,14), xlim=(35,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

In [44]:
scale = np.abs(np.mean(amps3[64:74]))/2
print("Scale is ", scale)

Scale is  898.018270301


### No phase response

In [76]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp(i):
    
#     print(i,'  ******************')
    fname = r'/data/old_trap/20210405/bead1/neutrality/wo_height_fb_50V_41_Hz_x/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.electrode_data[3])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     print(m1_tmp.values)
    phi_tmp = m1_tmp.values[2]
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [72]:
get_amp(1)

<ValueView of Minuit at 4d9ada28>
  A: 2.3989805697510747e-05
  f: 41.0
  phi: -1.5118696767779092


-48.77699363969441

In [78]:
amps_50V_wo_fb = []
for i in range(1000):
    print(i,'/1000')
    if (i!=700):
        if (i!=888):
            amps_50V_wo_fb.append(get_amp(i))

0 /1000
1 /1000
2 /1000
3 /1000
4 /1000
5 /1000
6 /1000
7 /1000
8 /1000
9 /1000
10 /1000
11 /1000
12 /1000
13 /1000
14 /1000
15 /1000
16 /1000
17 /1000
18 /1000
19 /1000
20 /1000
21 /1000
22 /1000
23 /1000
24 /1000
25 /1000
26 /1000
27 /1000
28 /1000
29 /1000
30 /1000
31 /1000
32 /1000
33 /1000
34 /1000
35 /1000
36 /1000
37 /1000
38 /1000
39 /1000
40 /1000
41 /1000
42 /1000
43 /1000
44 /1000
45 /1000
46 /1000
47 /1000
48 /1000
49 /1000
50 /1000
51 /1000
52 /1000
53 /1000
54 /1000
55 /1000
56 /1000
57 /1000
58 /1000
59 /1000
60 /1000
61 /1000
62 /1000
63 /1000
64 /1000
65 /1000
66 /1000
67 /1000
68 /1000
69 /1000
70 /1000
71 /1000
72 /1000
73 /1000
74 /1000
75 /1000
76 /1000
77 /1000
78 /1000
79 /1000
80 /1000
81 /1000
82 /1000
83 /1000
84 /1000
85 /1000
86 /1000
87 /1000
88 /1000
89 /1000
90 /1000
91 /1000
92 /1000
93 /1000
94 /1000
95 /1000
96 /1000
97 /1000
98 /1000
99 /1000
100 /1000
101 /1000
102 /1000
103 /1000
104 /1000
105 /1000
106 /1000
107 /1000
108 /1000
109 /1000
110 /1000


832 /1000
833 /1000
834 /1000
835 /1000
836 /1000
837 /1000
838 /1000
839 /1000
840 /1000
841 /1000
842 /1000
843 /1000
844 /1000
845 /1000
846 /1000
847 /1000
848 /1000
849 /1000
850 /1000
851 /1000
852 /1000
853 /1000
854 /1000
855 /1000
856 /1000
857 /1000
858 /1000
859 /1000
860 /1000
861 /1000
862 /1000
863 /1000
864 /1000
865 /1000
866 /1000
867 /1000
868 /1000
869 /1000
870 /1000
871 /1000
872 /1000
873 /1000
874 /1000
875 /1000
876 /1000
877 /1000
878 /1000
879 /1000
880 /1000
881 /1000
882 /1000
883 /1000
884 /1000
885 /1000
886 /1000
887 /1000
888 /1000
889 /1000
890 /1000
891 /1000
892 /1000
893 /1000
894 /1000
895 /1000
896 /1000
897 /1000
898 /1000
899 /1000
900 /1000
901 /1000
902 /1000
903 /1000
904 /1000
905 /1000
906 /1000
907 /1000
908 /1000
909 /1000
910 /1000
911 /1000
912 /1000
913 /1000
914 /1000
915 /1000
916 /1000
917 /1000
918 /1000
919 /1000
920 /1000
921 /1000
922 /1000
923 /1000
924 /1000
925 /1000
926 /1000
927 /1000
928 /1000
929 /1000
930 /1000
931 /1000


In [66]:
amps_50V_w_fb = np.array(amps_50V_w_fb)
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(amps_50V_w_fb)), amps_50V_w_fb/scale, label='50V, 41Hz., NoSpin')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
_,ax = plt.subplots()
data = amps_50V_w_fb/scale
data2 = amps_50V_wo_fb/scale
_ = ax.hist(data, bins=50, range=(-0.5,0.5), label='w/ fb')
_ = ax.hist(data2, bins=50, range=(-0.5,0.5), alpha=0.4, label='w/o fb')
ax.set(xlabel=r'$\epsilon$')
ax.legend()
np.mean(data), np.std(data)/np.sqrt(len(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.0064723724697288388, 0.0053179684403218026)

In [69]:
data = amps_50V_w_fb/scale
means = [np.mean(data[j*10:(j+1)*10]) for j in range(100)]
stds = [np.std(data[j*10:(j+1)*10]) for j in range(100)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,10), means, yerr=stds/np.sqrt(10),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [83]:
data = amps_50V_wo_fb/scale
means = [np.mean(data[j*10:(j+1)*10]) for j in range(100)]
stds = [np.std(data[j*10:(j+1)*10]) for j in range(100)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,10), means, yerr=stds/np.sqrt(10),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [70]:
data = amps_50V_w_fb/scale
means = [np.mean(data[j*100:(j+1)*100]) for j in range(10)]
stds = [np.std(data[j*100:(j+1)*100]) for j in range(10)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,1000,100), means, yerr=stds/np.sqrt(100),fmt='o')
ax.plot(np.arange(0,1000,100), np.arange(0,1000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]